# This notebook demonstrates successful data api call and processing



In [5]:
from components.AlpacaDataManager import *
from components.TrueBautistStrategy import *
import yaml

In [6]:
api_keys = './config/api_keys.yaml'
strategy  = "./config/stage-based-trading-system.yaml"


with open(api_keys, 'r') as file:
    keys = yaml.safe_load(file)

with open(strategy, 'r') as file:
    yaml_trade_config = yaml.safe_load(file)


strategy = TrueBautistStrategy(yaml_trade_config, keys)

In [7]:
strategy.config

{'symbols': ['PLTR', 'SOFI', 'JEPQ'],
 'timeframe': '1Min',
 'start_date': '2024-12-27',
 'end_date': '2025-01-27',
 'stage1_conditions': [{'indicator': 'close',
   'comparison': 'between',
   'value': ['lowerband', 'upperband']},
  {'indicator': 'close',
   'comparison': 'between',
   'value': ['sma_20', 'sma_50']},
  {'indicator': 'rsi', 'comparison': 'between', 'value': [40, 60]},
  {'indicator': 'atr', 'comparison': 'below', 'value': 1000}],
 'entry_conditions': [{'indicator': 'close',
   'comparison': 'above',
   'value': 'sma_20'},
  {'indicator': 'ema_5', 'comparison': 'crosses_above', 'value': 'sma_20'}],
 'exit_conditions': [{'indicator': 'close',
   'comparison': 'below',
   'value': 'ema_20'},
  {'indicator': 'ema_5', 'comparison': 'crosses_below', 'value': 'ema_20'},
  {'indicator': 'close', 'comparison': 'below', 'value': 'lowerband'}],
 'risk_management': {'position_sizing_method': 'risk_based',
  'risk_per_trade': 0.7,
  'stop_loss': 0.02,
  'take_profit': 0.06,
  'max_p

##### Data saved to strategy.df when calling get_data()

- get_data() processes all the indicators from the yaml config
- All the paramaters for each indicator are passed in the config.
- Need to implement the TrueBautistStrategy check_conditions method for entry and exit condition checking. - MIKE A. 

In [8]:
strategy.get_data()  # Load and prepare data including indicator calculations 


Getting data for these symbols:  ['PLTR', 'SOFI', 'JEPQ']


,timestamp,symbol,close,high,low,trades,open,volume,vwap,ema_5,...,close_prev,trades_prev,ema_5_prev,sma_20_prev,sma_50_prev,rsi_prev,upperband_prev,middleband_prev,lowerband_prev,atr_prev
0,2024-12-27 14:32:00+00:00,JEPQ,57.885,57.885,57.88,3,57.88,400,57.881250,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-12-27 14:33:00+00:00,JEPQ,57.890,57.890,57.89,2,57.89,300,57.890000,NaN,...,57.885,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-12-27 14:37:00+00:00,JEPQ,57.830,57.830,57.82,8,57.82,1090,57.828955,NaN,...,57.890,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-12-27 14:38:00+00:00,JEPQ,57.780,57.840,57.78,5,57.84,500,57.792000,NaN,...,57.830,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-12-27 14:40:00+00:00,JEPQ,57.780,57.780,57.78,5,57.78,300,57.780000,57.833000,...,57.780,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-12-30 20:35:00+00:00,SOFI,15.685,15.685,15.68,6,15.68,2247,15.682098,15.691225,...,15.675,25.0,15.694338,15.73775,15.7491,40.804680,15.799367,15.7491,15.698833,0.010841
9996,2024-12-30 20:36:00+00:00,SOFI,15.690,15.695,15.69,27,15.69,5559,15.690091,15.690817,...,15.685,6.0,15.691225,15.73350,15.7477,42.105871,15.801037,15.7477,15.694363,0.010824
9997,2024-12-30 20:37:00+00:00,SOFI,15.680,15.680,15.68,1,15.68,200,15.680000,15.687211,...,15.690,27.0,15.690817,15.72975,15.7464,42.747949,15.802079,15.7464,15.690721,0.010808
9998,2024-12-30 20:38:00+00:00,SOFI,15.670,15.670,15.67,7,15.67,2753,15.670000,15.681474,...,15.680,1.0,15.687211,15.72525,15.7446,41.801815,15.802870,15.7446,15.686330,0.010791


In [9]:
from lumibot.backtesting import PandasDataBacktesting, BacktestingBroker
from datetime import datetime
from lumibot.strategies import Strategy
from lumibot.traders import Trader
from dotenv import load_dotenv
import os
# importing the trader class
from lumibot.traders import Trader
# importing the alpaca broker class
from lumibot.brokers import Alpaca

2025-01-28 22:51:50 | lumibot.credentials | WARNING | No .env file found. This is ok if you are using environment variables or secrets (like on Replit, AWS, etc), but if you are not, please create a .env file in the root directory of the project.


In [10]:
ALPACA_CONFIG = {
    # Put your own Alpaca key here:
    "API_KEY": keys['api_key_paper'],
    # Put your own Alpaca secret here:
    "API_SECRET": keys['api_secret_paper'],
    # Set this to False to use a live account
    "PAPER": True
}

In [11]:
class YAMLStrategy(Strategy):
    def initialize(self):
        self.sleeptime = "1M"
        self.symbols = ["PLTR", "SOFI"]  # Define your symbols

    def on_trading_iteration(self):
        for symbol in self.symbols:
            print(symbol)
            # Get data for each symbol
            history = self.get_historical_prices(symbol, 30)
            position = self.get_position(symbol)
            
            print("histories: ",history)
            # Your trading logic for each symbol
            current_price = history.close[-1]
            sma = history.close.rolling(20).mean()
            
            if current_price > sma[-1] and not position:
                self.buy(symbol, 100)
            elif current_price < sma[-1] and position:
                self.sell_all(symbol)


##### Prepping data for the custom strategy


In [12]:
pltr_df = strategy.df.copy()
pltr_df = pltr_df[pltr_df.symbol == 'PLTR']
pltr_df = pltr_df[["timestamp","open","high","low","close", "volume"]]
pltr_df.set_index("timestamp")

,open,high,low,close,volume
timestamp,,,,,
2024-12-27 13:31:00+00:00,81.640,81.66,81.640,81.660,490
2024-12-27 14:12:00+00:00,81.410,81.41,81.410,81.410,100
2024-12-27 14:16:00+00:00,81.360,81.36,81.350,81.350,232
2024-12-27 14:18:00+00:00,81.200,81.20,81.200,81.200,130
2024-12-27 14:23:00+00:00,81.340,81.34,81.340,81.340,100
...,...,...,...,...,...
2025-01-27 20:55:00+00:00,75.150,75.32,75.150,75.255,10956
2025-01-27 20:56:00+00:00,75.245,75.33,75.245,75.330,10521
2025-01-27 20:57:00+00:00,75.325,75.45,75.310,75.380,10181


In [13]:
sofi_df = strategy.df.copy()
sofi_df = sofi_df[sofi_df.symbol == "SOFI"]
sofi_df = sofi_df[["timestamp","open","high","low","close", "volume"]]
sofi_df.set_index("timestamp")

,open,high,low,close,volume
timestamp,,,,,
2024-12-27 14:30:00+00:00,16.410,16.420,16.345,16.420,4070
2024-12-27 14:31:00+00:00,16.380,16.460,16.370,16.380,1203
2024-12-27 14:32:00+00:00,16.400,16.400,16.350,16.350,2534
2024-12-27 14:33:00+00:00,16.340,16.340,16.300,16.310,2904
2024-12-27 14:34:00+00:00,16.325,16.385,16.300,16.300,2232
...,...,...,...,...,...
2024-12-30 20:35:00+00:00,15.680,15.685,15.680,15.685,2247
2024-12-30 20:36:00+00:00,15.690,15.695,15.690,15.690,5559
2024-12-30 20:37:00+00:00,15.680,15.680,15.680,15.680,200


In [14]:
# handle the dataframe to lumi
from lumibot.entities import Asset
from lumibot.traders import Trader

pltr_asset = Asset(
    symbol="PLTR",
    asset_type=Asset.AssetType.STOCK,
)

from lumibot.entities import Data

pltr_data = Data(
    pltr_asset,
    pltr_df,
    timestep="minute",
)

In [15]:
pandas_data = {
    pltr_asset: pltr_data
}

In [20]:
from lumibot.example_strategies.stock_momentum import Momentum


doesn't work, trying Lumibots own strategy below

In [22]:
trader = Trader(backtest=True)
data_source = PandasDataBacktesting(
    pandas_data = pandas_data,
    datetime_start=datetime.strptime(strategy.config['start_date'], "%Y-%m-%d"), 
    datetime_end=datetime.strptime(strategy.config['end_date'], "%Y-%m-%d")
)
broker = BacktestingBroker(data_source)
strat  = Momentum(name="My Momentum Test", 
                             budget=10000, broker=broker, 
                             symbols=strategy.config['symbols'],
                             )

trader.add_strategy(strat)
trader.run_all()

Progress ||   1.28%  [Elapsed: 0:00:00 ETA: 0:01:11] 2025-01-28 22:57:25,102: ERROR: single positional indexer is out-of-bounds
2025-01-28 22:57:25,106: ERROR: [My Momentum Test] Traceback (most recent call last):
  File "c:\Users\fwmac\anaconda3\envs\alpaca\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 1060, in run
    self._run_trading_session()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\fwmac\anaconda3\envs\alpaca\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 1000, in _run_trading_session
    self._on_trading_iteration()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\fwmac\anaconda3\envs\alpaca\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 305, in func_output
    result = func_input(self, *args, **kwargs)
  File "c:\Users\fwmac\anaconda3\envs\alpaca\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 330, in func_output
    result = func_input(self, *args, **kwargs)
  File "c:\Users\fwmac\anaconda3\envs\a

Exception in thread My Momentum Test:
Traceback (most recent call last):
  File "c:\Users\fwmac\anaconda3\envs\alpaca\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 1060, in run
    self._run_trading_session()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\fwmac\anaconda3\envs\alpaca\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 1000, in _run_trading_session
    self._on_trading_iteration()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\fwmac\anaconda3\envs\alpaca\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 305, in func_output
    result = func_input(self, *args, **kwargs)
  File "c:\Users\fwmac\anaconda3\envs\alpaca\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 330, in func_output
    result = func_input(self, *args, **kwargs)
  File "c:\Users\fwmac\anaconda3\envs\alpaca\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 467, in _on_trading_iteration
    raise e
  File "c:\Users\fwmac\anaco

2025-01-28 22:57:25,298: ERROR: Not enough data to create a tearsheet, at least 2 days of data are required. Skipping


{'My Momentum Test': {}}

#### Momentum test from [Lumibot Github](https://github.com/Lumiwealth/lumibot/blob/dev/lumibot/example_strategies/stock_momentum.py)

In [36]:
from datetime import datetime

from lumibot.strategies.strategy import Strategy

"""
Strategy Description

Buys the best performing asset from self.symbols over self.period number of days.
For example, if SPY increased 2% yesterday, but VEU and AGG only increased 1% yesterday,
then we will buy SPY.
"""


class Momentum(Strategy):
    # =====Overloading lifecycle methods=============

    def initialize(self, symbols=None):
        # Setting the waiting period (in days)
        self.period = 2

        # The counter for the number of days we have been holding the current asset
        self.counter = 0

        # There is only one trading operation per day
        # No need to sleep between iterations
        self.sleeptime = 0

        # Set the symbols that we will be monitoring for momentum
        if symbols:
            self.symbols = symbols
        else:
            self.symbols = ["SPY", "VEU", "AGG"]

        # The asset that we want to buy/currently own, and the quantity
        self.asset = ""
        self.quantity = 0

    def on_trading_iteration(self):
        # When the counter reaches the desired holding period,
        # re-evaluate which asset we should be holding
        momentums = []
        if self.counter == self.period or self.counter == 0:
            self.counter = 0
            momentums = self.get_assets_momentums()

            # Get the asset with the highest return in our period
            # (aka the highest momentum)
            momentums.sort(key=lambda x: x.get("return"))
            best_asset_data = momentums[-1]
            best_asset = best_asset_data["symbol"]
            best_asset_return = best_asset_data["return"]

            # Get the data for the currently held asset
            if self.asset:
                current_asset_data = [
                    m for m in momentums if m["symbol"] == self.asset
                ][0]
                current_asset_return = current_asset_data["return"]

                # If the returns are equals, keep the current asset
                if current_asset_return >= best_asset_return:
                    best_asset = self.asset
                    best_asset_data = current_asset_data

            self.log_message("%s best symbol." % best_asset)

            # If the asset with the highest momentum has changed, buy the new asset
            if best_asset != self.asset:
                # Sell the current asset that we own
                if self.asset:
                    self.log_message("Swapping %s for %s." % (self.asset, best_asset))
                    order = self.create_order(self.asset, self.quantity, "sell")
                    self.submit_order(order)

                # Calculate the quantity and send the buy order for the new asset
                self.asset = best_asset
                best_asset_price = best_asset_data["price"]
                self.quantity = int(self.portfolio_value // best_asset_price)
                order = self.create_order(self.asset, self.quantity, "buy")
                self.submit_order(order)
            else:
                self.log_message("Keeping %d shares of %s" % (self.quantity, self.asset))

        self.counter += 1

        # Stop for the day, since we are looking at daily momentums
        self.await_market_to_close()

    def on_abrupt_closing(self):
        # Sell all positions
        self.sell_all()

    def trace_stats(self, context, snapshot_before):
        """
        Add additional stats to the CSV logfile
        """
        # Get the values of all our variables from the last iteration
        row = {
            "old_best_asset": snapshot_before.get("asset"),
            "old_asset_quantity": snapshot_before.get("quantity"),
            "old_cash": snapshot_before.get("cash"),
            "new_best_asset": self.asset,
            "new_asset_quantity": self.quantity,
        }

        # Get the momentums of all the assets from the context of on_trading_iteration
        # (notice that on_trading_iteration has a variable called momentums, this is what
        # we are reading here)
        momentums = context.get("momentums")
        if len(momentums) != 0:
            for item in momentums:
                symbol = item.get("symbol")
                for key in item:
                    if key != "symbol":
                        row[f"{symbol}_{key}"] = item[key]

        # Add all of our values to the row in the CSV file. These automatically get
        # added to portfolio_value, cash and return
        return row

    # =============Helper methods====================

    def get_assets_momentums(self):
        """
        Gets the momentums (the percentage return) for all the assets we are tracking,
        over the time period set in self.period
        """
        momentums = []
        data = self.get_bars(self.symbols, self.period + 2, timestep="day")
        for asset, bars_set in data.items():
            # Get the return for symbol over self.period days
            symbol = asset.symbol
            symbol_momentum = bars_set.get_momentum(num_periods=self.period)
            self.log_message(
                "%s has a return value of %.2f%% over the last %d day(s)."
                % (symbol, 100 * symbol_momentum, self.period)
            )

            momentums.append(
                {
                    "symbol": symbol,
                    "price": bars_set.get_last_price(),
                    "return": symbol_momentum,
                }
            )

        return momentums




In [37]:

is_live = False

if is_live:
    from lumibot.credentials import ALPACA_CONFIG
    from lumibot.brokers import Alpaca

    broker = Alpaca(ALPACA_CONFIG)

    strategy = Momentum(broker=broker)
    strategy.run_live()

else:
    from lumibot.backtesting import YahooDataBacktesting

    # Backtest this strategy
    backtesting_start = datetime(2023, 1, 1)
    backtesting_end = datetime(2023, 8, 1)

    results = Momentum.backtest(
        YahooDataBacktesting,
        backtesting_start,
        backtesting_end,
        benchmark_asset="SPY",
    )

Progress || 100.00%  [Elapsed: 0:00:40 ETA: 0:00:00] 